# Customer Clustering Segmentation Notebook
Dataset from Kaggle: **Download HERE** https://www.kaggle.com/datasets/shivamb/bank-customer-segmentation

**Objectives:**

- Split and group customers based on their profiles with an unsupervised learning algorithm.

## Import dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 


In [20]:
df = pd.read_csv('bank_transactions.csv')
df

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5
...,...,...,...,...,...,...,...,...,...
1048562,T1048563,C8020229,8/4/90,M,NEW DELHI,7635.19,18/9/16,184824,799.0
1048563,T1048564,C6459278,20/2/92,M,NASHIK,27311.42,18/9/16,183734,460.0
1048564,T1048565,C6412354,18/5/89,M,HYDERABAD,221757.06,18/9/16,183313,770.0
1048565,T1048566,C6420483,30/8/78,M,VISAKHAPATNAM,10117.87,18/9/16,184706,1000.0


## Preprocessing

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048567 entries, 0 to 1048566
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   TransactionID            1048567 non-null  object 
 1   CustomerID               1048567 non-null  object 
 2   CustomerDOB              1045170 non-null  object 
 3   CustGender               1047467 non-null  object 
 4   CustLocation             1048416 non-null  object 
 5   CustAccountBalance       1046198 non-null  float64
 6   TransactionDate          1048567 non-null  object 
 7   TransactionTime          1048567 non-null  int64  
 8   TransactionAmount (INR)  1048567 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 72.0+ MB


In [4]:
df.columns

Index(['TransactionID', 'CustomerID', 'CustomerDOB', 'CustGender',
       'CustLocation', 'CustAccountBalance', 'TransactionDate',
       'TransactionTime', 'TransactionAmount (INR)'],
      dtype='object')

In [5]:
df.isnull().sum()

TransactionID                 0
CustomerID                    0
CustomerDOB                3397
CustGender                 1100
CustLocation                151
CustAccountBalance         2369
TransactionDate               0
TransactionTime               0
TransactionAmount (INR)       0
dtype: int64

In [6]:
df.dropna(inplace=True)
df

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5
...,...,...,...,...,...,...,...,...,...
1048562,T1048563,C8020229,8/4/90,M,NEW DELHI,7635.19,18/9/16,184824,799.0
1048563,T1048564,C6459278,20/2/92,M,NASHIK,27311.42,18/9/16,183734,460.0
1048564,T1048565,C6412354,18/5/89,M,HYDERABAD,221757.06,18/9/16,183313,770.0
1048565,T1048566,C6420483,30/8/78,M,VISAKHAPATNAM,10117.87,18/9/16,184706,1000.0


**Transform column "CustomerDOB" into numerical values -> Age:**

In [7]:
from datetime import datetime, date
date.today().year

2022

In [8]:
from datetime import datetime, date

today = date.today()
df.CustomerDOB = df.CustomerDOB.astype(str) 
df['year_birth'] = df.CustomerDOB.str[-2:]
df['year_birth'] = '19'+df['year_birth']
df['year_birth'] = df['year_birth'].astype(int)
df['age'] = today.year - df['year_birth']
df

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),year_birth,age
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0,1994,28
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0,1957,65
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0,1996,26
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0,1973,49
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5,1988,34
...,...,...,...,...,...,...,...,...,...,...,...
1048562,T1048563,C8020229,8/4/90,M,NEW DELHI,7635.19,18/9/16,184824,799.0,1990,32
1048563,T1048564,C6459278,20/2/92,M,NASHIK,27311.42,18/9/16,183734,460.0,1992,30
1048564,T1048565,C6412354,18/5/89,M,HYDERABAD,221757.06,18/9/16,183313,770.0,1989,33
1048565,T1048566,C6420483,30/8/78,M,VISAKHAPATNAM,10117.87,18/9/16,184706,1000.0,1978,44


**Convert age into new category:**

In [9]:
print(df.age.min(),df.age.max())
# teenagers => age = 18 to 30
# adults => age = 31 to 50
# mature => age = 51 to 70
# elder => age > 71

23 122


In [10]:
df['age_classification'] = 0
df.loc[df.age<=30 ,'age_classification']='teen'
df.loc[(df.age>30) & (df.age<=50),'age_classification']='adults'
df.loc[(df.age>50) & (df.age<=70) ,'age_classification']='mature'
df.loc[(df.age>70), 'age_classification']='elder'

df

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),year_birth,age,age_classification
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0,1994,28,teen
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0,1957,65,mature
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0,1996,26,teen
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0,1973,49,adults
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5,1988,34,adults
...,...,...,...,...,...,...,...,...,...,...,...,...
1048562,T1048563,C8020229,8/4/90,M,NEW DELHI,7635.19,18/9/16,184824,799.0,1990,32,adults
1048563,T1048564,C6459278,20/2/92,M,NASHIK,27311.42,18/9/16,183734,460.0,1992,30,teen
1048564,T1048565,C6412354,18/5/89,M,HYDERABAD,221757.06,18/9/16,183313,770.0,1989,33,adults
1048565,T1048566,C6420483,30/8/78,M,VISAKHAPATNAM,10117.87,18/9/16,184706,1000.0,1978,44,adults


### Use one hot encoding technique to conver categorical variables to binary variables and append them to the feature Data Frame


In [11]:
from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(df)
categorical_columns


['TransactionID',
 'CustomerID',
 'CustomerDOB',
 'CustGender',
 'CustLocation',
 'TransactionDate',
 'age_classification']

In [12]:
data_categorical = df[categorical_columns]
data_categorical.drop(columns=['TransactionID','CustLocation','TransactionDate','CustomerID','CustomerDOB'],inplace=True)
data_categorical.head()

C:\Users\Maxter\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,CustGender,age_classification
0,F,teen
1,M,mature
2,F,teen
3,F,adults
4,F,adults


In [13]:
Feature = df[["CustomerID",'CustGender', 'CustAccountBalance', 'TransactionAmount (INR)','age_classification']]
Feature = pd.concat([Feature,pd.get_dummies(data_categorical)], axis=1)
Feature.head()

,CustomerID,CustGender,CustAccountBalance,TransactionAmount (INR),age_classification,CustGender_F,CustGender_M,CustGender_T,age_classification_adults,age_classification_elder,age_classification_mature,age_classification_teen
0,C5841053,F,17819.05,25.0,teen,1,0,0,0,0,0,1
1,C2142763,M,2270.69,27999.0,mature,0,1,0,0,0,1,0
2,C4417068,F,17874.44,459.0,teen,1,0,0,0,0,0,1
3,C5342380,F,866503.21,2060.0,adults,1,0,0,1,0,0,0
4,C9031234,F,6714.43,1762.5,adults,1,0,0,1,0,0,0


### Grouping by CustomerID show customer's behavior

In [14]:
Feature['transactions']=1
df_customer = Feature.groupby(['CustomerID'],as_index=False).agg({'CustAccountBalance':np.mean, 'TransactionAmount (INR)':np.sum,'transactions':np.sum,
                                                             'CustGender_F':np.sum,"CustGender_M":np.sum,"CustGender_T":np.sum,
                                                             "age_classification_adults":np.sum,"age_classification_elder":np.sum,
                                                             "age_classification_mature":np.sum,"age_classification_teen":np.sum})
df_customer.rename(columns={"CustAccountBalance":"AverageCustAccountBalance"}, inplace=True)
df_customer['medium_transaction_value'] = df_customer['TransactionAmount (INR)']/df_customer['transactions']
print(df_customer.info())
df_customer


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879358 entries, 0 to 879357
Data columns (total 12 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   CustomerID                 879358 non-null  object 
 1   AverageCustAccountBalance  879358 non-null  float64
 2   TransactionAmount (INR)    879358 non-null  float64
 3   transactions               879358 non-null  int64  
 4   CustGender_F               879358 non-null  uint8  
 5   CustGender_M               879358 non-null  uint8  
 6   CustGender_T               879358 non-null  uint8  
 7   age_classification_adults  879358 non-null  uint8  
 8   age_classification_elder   879358 non-null  uint8  
 9   age_classification_mature  879358 non-null  uint8  
 10  age_classification_teen    879358 non-null  uint8  
 11  medium_transaction_value   879358 non-null  float64
dtypes: float64(3), int64(1), object(1), uint8(7)
memory usage: 39.4+ MB
None


,CustomerID,AverageCustAccountBalance,TransactionAmount (INR),transactions,CustGender_F,CustGender_M,CustGender_T,age_classification_adults,age_classification_elder,age_classification_mature,age_classification_teen,medium_transaction_value
0,C1010011,76340.635,5106.0,2,1,1,0,1,0,0,1,2553.0
1,C1010012,24204.490,1499.0,1,0,1,0,0,0,0,1,1499.0
2,C1010014,100112.950,1455.0,2,1,1,0,1,0,0,1,727.5
3,C1010018,496.180,30.0,1,1,0,0,1,0,0,0,30.0
4,C1010024,87058.650,5000.0,1,0,1,0,0,0,1,0,5000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
879353,C9099836,133067.230,691.0,1,0,1,0,1,0,0,0,691.0
879354,C9099877,96063.460,222.0,1,0,1,0,0,0,0,1,222.0
879355,C9099919,5559.750,126.0,1,0,1,0,0,0,0,1,126.0
879356,C9099941,35295.920,50.0,1,0,1,0,0,0,0,1,50.0


### Scaling process

In [15]:
from sklearn.preprocessing import StandardScaler
X = df_customer.drop('CustomerID', axis=1)
X = np.nan_to_num(X)
Clus_dataSet = StandardScaler().fit_transform(X)
Clus_dataSet

array([[-0.04790311,  0.45225274,  1.81640799, ..., -0.28241254,
         1.7734225 ,  0.15329836],
       [-0.11298516, -0.0495341 , -0.41099187, ..., -0.28241254,
         1.7734225 , -0.0105381 ],
       [-0.0182279 , -0.05565515,  1.81640799, ..., -0.28241254,
         1.7734225 , -0.13046204],
       ...,
       [-0.13625957, -0.24053863, -0.41099187, ..., -0.28241254,
         1.7734225 , -0.22396074],
       [-0.09913962, -0.25111135, -0.41099187, ..., -0.28241254,
         1.7734225 , -0.23577437],
       [-0.13450048, -0.13912399, -0.41099187, ..., -0.28241254,
        -0.51793245, -0.11064311]])

### Modeling with KMeans Algorithm

In [16]:
np.random.seed(66)
from sklearn.cluster import KMeans

clusterNum = 5
k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 4)
k_means.fit(Clus_dataSet)
labels = k_means.labels_
print(labels)

[3 0 3 ... 0 0 2]


### Customer Behavior Segmentation with Kmeans

In [17]:
df_customer["cluster"] = labels
df_customer["customers"]=1
df_customer_cluster=df_customer.groupby('cluster',as_index=False).agg({'AverageCustAccountBalance':np.sum,'TransactionAmount (INR)':np.sum,
                                                        'transactions':np.sum,'medium_transaction_value':np.mean,'customers':np.sum,
                                                        'CustGender_F':np.sum,"CustGender_M":np.sum,"CustGender_T":np.sum,
                                                        "age_classification_adults":np.sum,"age_classification_elder":np.sum,
                                                        "age_classification_mature":np.sum,"age_classification_teen":np.sum})
df_customer_cluster["AverageCustAccountBalance/customers"] = df_customer_cluster['AverageCustAccountBalance']/df_customer_cluster['customers']
df_customer_cluster["TransactionAmount/customers"] = df_customer_cluster['TransactionAmount (INR)']/df_customer_cluster['customers']
df_customer_cluster["transactions/customers"] = df_customer_cluster['transactions']/df_customer_cluster['customers']
df_customer_cluster.rename(columns={'AverageCustAccountBalance':'AccumulatedClusterBalance','TransactionAmount (INR)':'AccumulatedTransactionAmount'}, inplace=True)
df_customer_cluster

,cluster,AccumulatedClusterBalance,AccumulatedTransactionAmount,transactions,medium_transaction_value,customers,CustGender_F,CustGender_M,CustGender_T,age_classification_adults,age_classification_elder,age_classification_mature,age_classification_teen,AverageCustAccountBalance/customers,TransactionAmount/customers,transactions/customers
0,0,4.940459e+09,1.153132e+08,154691,735.208375,147546,49788.0,104903.0,0,0.0,1125.0,1237.0,152329.0,3.348420e+04,781.540733,1.048426
1,1,1.986453e+10,2.663869e+08,171024,1562.243498,162271,171023.0,0.0,1,145501.0,8062.0,14300.0,3161.0,1.224158e+05,1641.617134,1.053941
2,2,5.669939e+10,5.939137e+08,440295,1348.899409,440295,0.0,440295.0,0,368623.0,37953.0,33719.0,0.0,1.287759e+05,1348.899409,1.000000
3,3,1.465670e+10,3.686749e+08,270943,1359.834565,125446,58703.0,212240.0,0,194336.0,17032.0,16715.0,42860.0,1.168367e+05,2938.912811,2.159838
4,4,4.714487e+09,2.869790e+08,4661,66128.852211,3800,1121.0,3540.0,0,2444.0,1120.0,679.0,418.0,1.240655e+06,75520.793929,1.226579


In [18]:
columns_toExplore = ['AccumulatedClusterBalance', 'AccumulatedTransactionAmount',
       'transactions', 'medium_transaction_value', 'customers', 'CustGender_F',
       'CustGender_M', 'CustGender_T', 'age_classification_adults',
       'age_classification_elder', 'age_classification_mature',
       'age_classification_teen', 'AverageCustAccountBalance/customers',
       'TransactionAmount/customers', 'transactions/customers']


for column in columns_toExplore:
    print(70*"_")
    print(f"Ranking Cluster by {column}: ")
    dataframe = df_customer_cluster[['cluster', column]]
    rank = dataframe.sort_values(by=column, ascending = False)
    rank_df = pd.DataFrame(rank)
    champion = rank_df['cluster'].index
    champion_id = champion[0]
    looser = rank_df['cluster'].index
    looser_id = looser[clusterNum-1]
    print(36*"_")
    print(f"|  Champion is {champion_id}  |   Looser is {looser_id}  |")
    print(5*"_",5*'*',5*"_")
    print(rank_df)
    print(70*"*")
    print(" ")

______________________________________________________________________
Ranking Cluster by AccumulatedClusterBalance: 
____________________________________
|  Champion is 2  |   Looser is 4  |
_____ ***** _____
   cluster  AccumulatedClusterBalance
2        2               5.669939e+10
1        1               1.986453e+10
3        3               1.465670e+10
0        0               4.940459e+09
4        4               4.714487e+09
**********************************************************************
 
______________________________________________________________________
Ranking Cluster by AccumulatedTransactionAmount: 
____________________________________
|  Champion is 2  |   Looser is 0  |
_____ ***** _____
   cluster  AccumulatedTransactionAmount
2        2                  5.939137e+08
3        3                  3.686749e+08
4        4                  2.869790e+08
1        1                  2.663869e+08
0        0                  1.153132e+08
******************************

# Insights and Considerations:

# Cluster 0

**Accumulated Cluster Metrics:** 
Balance: MEDIUM | TransactionAmount: LOW | transactions: MEDIUM

medium_transaction_value: LOWEST | customers: MEDIUM

-------------

**Customer Characteristics:** 
Gender: Mixed, MAINLY MALE | Age: MAJORITY of TEENS are in this cluster. It has NO ADULTS.

AverageBalance/customers: LOWEST | TransactionAmount/customers: LOWEST | transactions/customers: LOW

# Cluster 1

**Accumulated Cluster Metrics:** 
Balance: HIGH | TransactionAmount: LOW | transactions: MEDIUM 

medium_transaction_value: MEDIUM | customers: MEDIUM

-----------------

**Customer Characteristics:** 
Gender: ONLY FEMALE | Age: mixed, but MAINLY ADULTS.

AverageBalance/customers: MEDIUM | TransactionAmount/customers: MEDIUM | transactions/customers: MEDIUM

# Cluster 2

**Accumulated Cluster Metrics:** 
Balance: HIGHEST | TransactionAmount: HIGHEST | transactions: HIGHEST 

medium_transaction_value: LOW | customers: HIGHEST

-----------------

**Customer Characteristics:** 
Gender: ONLY MALE | Age: HIGHEST Adults, Mature and Elder. It has NO Teenagers

AverageBalance/customers: HIGH | TransactionAmount/customers: LOW | transactions/customers: LOWEST

# Cluster 3

**Accumulated Cluster Metrics:** 
Balance: MEDIUM | TransactionAmount: HIGH | transactions: HIGH 

medium_transaction_value: MEDIUM | customers: MEDIUM

-----------------

**Customer Characteristics:** 
Gender: mixed, but MAINLY MALE | Age: mixed, but MAINLY ADULTS.

AverageBalance/customers: LOW | TransactionAmount/customers: HIGH | transactions/customers: HIGHEST

# Cluster 4

**Accumulated Cluster Metrics:** 
Balance: LOWEST | TransactionAmount: MEDIUM | transactions: LOWEST 

medium_transaction_value: HIGHEST | customers: LOWEST

-----------------

**Customer Characteristics:** 
Gender: mixed, but MAINLY MALE | Age: mixed, but MAINLY ADULTS.

AverageBalance/customers: HIGHEST | TransactionAmount/customers: HIGHEST | transactions/customers: MEDIUM